In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

### Test Local LangGraph Agent

In [ ]:
from langgraph_sdk import get_client

url = "https://leetmock-ts-fa225f46565756e7b0567441810f232f.default.us.langgraph.app"
client = get_client(url=url)

# Using the graph deployed with the name "agent"
assistant_id = "code-mock-staged-v1"

# create thread
thread = await client.threads.create()

print(thread)

In [2]:
openai_assistant = await client.assistants.create(graph_id=assistant_id)

In [ ]:
openai_assistant

In [1]:
from agent_graph.code_mock_staged_v1.graph import create_compiled_graph
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

# graph = create_graph().compile()
graph = create_compiled_graph()
config = { "configurable": {"thread_id": "1", "session_id1": "abc"} }

In [ ]:
async for chunk in graph.astream(
    input={
        "messages": ["Hi"],
        "event": "reminder",
        # "trigger": True,
    },
    config=config,
    stream_mode=["values"],
):
    print(chunk)

async for chunk in graph.astream(
    input={
        "messages": ["Hi"],
        # "event": "user_message"
    },
    config=config,
    stream_mode=["values"],
):
    print(chunk)

In [ ]:
from langchain_core.load.load import load, loads
from langchain_core.load.dump import dumpd, dumps
from langgraph.types import StateSnapshot

state = graph.get_state(config=config)
state

In [ ]:
dumped_state = dumps(state)
dumped_state

In [ ]:
loaded_state = loads(dumped_state, valid_namespaces=["agent_graph"])
loaded_state_snapshot = StateSnapshot(*loaded_state)
loaded_state_snapshot

In [ ]:
loaded_state_snapshot == state, dumpd(loaded_state_snapshot) == dumpd(state)

In [ ]:
loaded_state_snapshot.values == state.values

In [14]:
history = list(graph.get_state_history(config=config))

In [ ]:
dumped_history = dumps(history)
dumped_history

In [ ]:
loaded_history = loads(dumped_history, valid_namespaces=["agent_graph"])
loaded_history_snapshot = [StateSnapshot(*h) for h in loaded_history]
loaded_history_snapshot

In [ ]:
loaded_history_snapshot == history, dumpd(loaded_history_snapshot) == dumpd(history)

In [ ]:
dumped_state_byte = dumped_state.encode()
dumped_history_byte = dumped_history.encode()
print(f"State: {len(dumped_state_byte) / 1024:.2f} KB")
print(f"State History: {len(dumped_history_byte) / 1024:.2f} KB")

In [ ]:
!pip install graphviz

In [ ]:
a = graph.get_graph().draw_mermaid()

In [7]:
with open("graph.md", "w") as f:
    f.write(a)

In [ ]:
from pydantic.v1 import BaseModel, Field, PrivateAttr


class User(BaseModel):
    name: str
    age: int
    _a: str = PrivateAttr(...)

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        print("base class init")
        self._a = "a"


class User2(User):

    c: str = Field(...)


user = User2(name="Charlie", age=20, c="c")
print(user)  # Output: 50

In [33]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

CACHE_CONFIG = {"cache_control": {"type": "ephemeral"}}

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            template="You are a {role} assistant." * 1000,
            additional_kwargs=CACHE_CONFIG,
        ),
        HumanMessagePromptTemplate.from_template(
            template="{input}",
            additional_kwargs=CACHE_CONFIG,
        ),
    ]
)

chat = ChatAnthropic(
    model_name="claude-3-5-haiku-20241022",
    extra_headers={"anthropic-beta": "prompt-caching-2024-07-31"},  # type: ignore
)

openai = ChatOpenAI(name="gpt-4o-mini")

chain = prompt | chat
# chain.invoke(
#     {
#         "role": "helpful",
#         "input": "What is second page of the shakespeare?",
#     }
# ).dict()


/Users/byin/Documents/GitHub/leetmock-monorepo/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [45]:
openai.invoke(
    [
        SystemMessage(
            content=[
                {
                    "text": "You are a helpful assistant." * 1000,
                    "type": "text",
                    **CACHE_CONFIG,
                }
            ],
        ),
        HumanMessage(
            content=[
                {
                    "text": "What is second page of the shakespeare?",
                    "type": "text",
                    **CACHE_CONFIG,
                }
            ],
        ),
        HumanMessage(
            content=[
                {
                    "text": "What is first page of the shakespeare?",
                    "type": "text",
                    **CACHE_CONFIG,
                }
            ],
        ),
    ]
).dict()

{'content': "William Shakespeare was an English playwright and poet who lived in the late 16th and early 17th centuries. His works are widely regarded as some of the greatest in the English language. Shakespeare's first page would vary depending on the specific edition or collection of his works that you are referring to, as his plays and poems have been published and organized in many different ways over the centuries. If you have a specific edition in mind, I can help you locate the first page of Shakespeare's works in that edition.",
 'additional_kwargs': {'refusal': None},
 'response_metadata': {'token_usage': {'completion_tokens': 103,
   'prompt_tokens': 5034,
   'total_tokens': 5137,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 0,
    'rejected_prediction_tokens': 0},
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
  'model_name': 'gpt-3.5-turbo-0125',
  'system_fingerprint': None,
  'finis

In [1]:
import json
import time


json.dumps(
    {
        "context": "test",
        "name": "test",
        "time": time.time(),
    }
)


'{"context": "test", "name": "test", "time": 1731480108.65847}'